In [1]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from nltk.corpus import stopwords
import string
import re
from bs4 import BeautifulSoup
import torch
import nltk
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

/home/emil/Desktop/diplomski/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/emil/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
df_fake = pd.read_csv("input_data/Fake.csv")
df_true = pd.read_csv("input_data/True.csv")

In [4]:
df_fake["label"] = 0
df_true["label"] = 1

In [5]:
# Removing last 10 rows for manual testing
df_fake_manual_testing = df_fake.tail(50)
for i in range(23480,23430,-1):
    df_fake.drop([i], axis = 0, inplace = True)
    
    
df_true_manual_testing = df_true.tail(50)
for i in range(21416,21366,-1):
    df_true.drop([i], axis = 0, inplace = True)

In [6]:
df_fake_manual_testing["label"] = 0
df_true_manual_testing["label"] = 1

/tmp/ipykernel_46126/3685789944.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fake_manual_testing["label"] = 0
/tmp/ipykernel_46126/3685789944.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_true_manual_testing["label"] = 1


In [7]:
df_manual_testing = pd.concat([df_fake_manual_testing,df_true_manual_testing], axis = 0)
df_manual_testing.to_csv("manual_testing.csv")

In [8]:
df = pd.concat([df_fake, df_true], axis =0 )

In [9]:
df.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [10]:
len(df)

44798

In [11]:
df.isnull().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [12]:
df.dropna(inplace=True)

In [13]:
df.isnull().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [14]:
len(df)

44798

In [16]:
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

def preprocess_text(text):

    def remove_reuters_prefix(text):
        pattern = r'^[\s\S]*?\(reuters\) - '
        return re.sub(pattern, '', text)

    def strip_html(text):
        soup = BeautifulSoup(text, "html.parser")
        return soup.get_text()

    def remove_square_brackets(text):
        return re.sub('\[[^]]*\]', '', text)

    def remove_urls(text):
        return re.sub(r'http\S+', '', text)

    def remove_stopwords(text):
        final_text = []
        for i in text.split():
            if i.strip().lower() not in stop:
                final_text.append(i.strip())
        return " ".join(final_text)

    text = text.lower()
    text = remove_reuters_prefix(text)
    text = strip_html(text)
    text = remove_square_brackets(text)
    text = remove_urls(text)
    text = remove_stopwords(text)

    return text


In [17]:
df['text'] = df['text'].apply(preprocess_text)

/tmp/ipykernel_46126/2048953883.py:12: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")
/tmp/ipykernel_46126/2048953883.py:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [18]:
df.isnull().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [19]:
df = df.filter(items=['text', 'label'])

In [20]:
df.head()

,text,label
0,donald trump wish americans happy new year lea...,0
1,house intelligence committee chairman devin nu...,0
2,"friday, revealed former milwaukee sheriff davi...",0
3,"christmas day, donald trump announced would ba...",0
4,pope francis used annual christmas day message...,0


In [21]:
df = df.sample(frac = 1)
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [22]:
df.head()

,text,label
0,black protesters white police would killed sec...,0
1,colombia armed forces authorized launch air ra...,1
2,great idea! sheriff joe arpaio considering run...,0
3,share everyone! good reason (with exception ob...,0
4,opponents proposal create u.s. border tax impo...,1


In [15]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [23]:
def tokenize_function(data):
    return tokenizer(data['text'], padding="max_length", truncation=True)

In [24]:
dataset = Dataset.from_pandas(df)

In [25]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 44798/44798 [02:56<00:00, 254.28 examples/s]


In [26]:
train_test_split = tokenized_datasets.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [27]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [28]:
device

'cuda'

In [29]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [31]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [32]:
training_args = TrainingArguments(
    output_dir='./results',  # Directory for saving model checkpoints and other results
    num_train_epochs=3,  # Number of training epochs
    per_device_train_batch_size=8,  # Batch size for training
    per_device_eval_batch_size=8,  # Batch size for evaluation
    warmup_steps=500,  # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,  # Weight decay for regularization
    logging_dir='./logs',  # Directory for storing logs
    logging_steps=50,  # Log every 50 steps
    save_total_limit=3,  # Limit the total amount of checkpoints
    save_steps=2_000,  # Save checkpoint every 2,000 steps
    evaluation_strategy="steps",  # Evaluation strategy to use
    eval_steps=2_000,  # Evaluate every 2,000 steps
    load_best_model_at_end=True,  # Load the best model at the end of training
    metric_for_best_model="f1",  # Use F1 score to determine the best model
)

/home/emil/Desktop/diplomski/.venv/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [33]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

In [34]:
trainer.train()

                                                      
 15%|█▍        | 2000/13440 [27:08<2:18:30,  1.38it/s]

{'eval_loss': 0.011703373864293098, 'eval_accuracy': 0.9969866071428571, 'eval_precision': 0.9969869197785193, 'eval_recall': 0.9969866071428571, 'eval_f1': 0.9969866514822618, 'eval_runtime': 235.8809, 'eval_samples_per_second': 37.985, 'eval_steps_per_second': 4.748, 'epoch': 0.45}


 15%|█▌        | 2050/13440 [27:46<2:18:53,  1.37it/s]  

{'loss': 0.0266, 'grad_norm': 0.04126088693737984, 'learning_rate': 4.401081916537868e-05, 'epoch': 0.46}


 16%|█▌        | 2100/13440 [28:23<2:19:07,  1.36it/s]

...

100%|█████████▉| 13400/13440 [3:03:24<00:28,  1.41it/s]

{'loss': 0.0, 'grad_norm': 0.0006188718252815306, 'learning_rate': 1.5455950540958268e-07, 'epoch': 2.99}


100%|██████████| 13440/13440 [3:03:52<00:00,  1.22it/s]

{'train_runtime': 11032.7217, 'train_samples_per_second': 9.745, 'train_steps_per_second': 1.218, 'train_loss': 0.023050309770903932, 'epoch': 3.0}


TrainOutput(global_step=13440, training_loss=0.023050309770903932, metrics={'train_runtime': 11032.7217, 'train_samples_per_second': 9.745, 'train_steps_per_second': 1.218, 'total_flos': 2.828812200597504e+16, 'train_loss': 0.023050309770903932, 'epoch': 3.0})

In [35]:
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

100%|██████████| 1120/1120 [03:49<00:00,  4.88it/s]

Evaluation results: {'eval_loss': 0.0038314524572342634, 'eval_accuracy': 0.9994419642857143, 'eval_precision': 0.9994421721491528, 'eval_recall': 0.9994419642857143, 'eval_f1': 0.9994419559488603, 'eval_runtime': 229.7196, 'eval_samples_per_second': 39.004, 'eval_steps_per_second': 4.876, 'epoch': 3.0}


In [36]:
model.save_pretrained("./fake_news_classifier")
tokenizer.save_pretrained("./fake_news_classifier")

('./fake_news_classifier/tokenizer_config.json',
 './fake_news_classifier/special_tokens_map.json',
 './fake_news_classifier/vocab.txt',
 './fake_news_classifier/added_tokens.json')